**Table of contents**<a id='toc0_'></a>    
- [Initialize TSDataset, Pipeline, Model, Validator, Strategy](#toc1_1_)    
    - [TSDataset](#toc1_1_1_)    
    - [Pipeline](#toc1_1_2_)    
    - [Trainer](#toc1_1_3_)    
    - [Strategy](#toc1_1_4_)    
- [Save and load checkpoints](#toc2_)    
    - [Save checkpoint](#toc2_1_1_)    
  - [Load checkpoint for finetune](#toc2_2_)    
  - [Load checkpoint for inference](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import warnings

warnings.filterwarnings("ignore")

from pathlib import Path
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import torch

from tsururu.dataset import Pipeline, TSDataset
from tsururu.model_training import DLTrainer
from tsururu.model_training import KFoldCrossValidator
from tsururu.models import DLinear_NN
from tsururu.strategies import MIMOStrategy
from tsururu.transformers import (
    DateSeasonsGenerator,
    LagTransformer,
    SequentialTransformer,
    TargetGenerator,
    UnionTransformer,
)

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(value, idx_fold)
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

## <a id='toc1_1_'></a>[Initialize TSDataset, Pipeline, Model, Validator, Strategy](#toc0_)

The initialization of the main components is exactly the same as when using ML models. The only difference is that `DLTrainer` allows you to pass many more parameters compared to `MLTrainer`.

### <a id='toc1_1_1_'></a>[TSDataset](#toc0_)

In [3]:
df_path = Path("../datasets/global/simulated_data_to_check.csv")

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continuous",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


### <a id='toc1_1_2_'></a>[Pipeline](#toc0_)

In [5]:
lag = LagTransformer(lags=7)
target_generator = TargetGenerator()
date_features = DateSeasonsGenerator()
date_lags = LagTransformer(lags=7)

union_1 = UnionTransformer(transformers_list=[lag, target_generator])
seq_1 = SequentialTransformer(transformers_list=[union_1], input_features=["value"])
seq_2 = SequentialTransformer(transformers_list=[date_features, date_lags], input_features=["date"])
union = UnionTransformer(transformers_list=[seq_1, seq_2])

pipeline = Pipeline(union, multivariate=False)

### <a id='toc1_1_3_'></a>[Trainer](#toc0_)

Currently, the available architectures are `DLinear`, `PatchTST`, `GPT2`, `CycleNet`, `TimesNet`. Moreover, adding your own architecture is quite simple if you follow the logic of the base model class.

In [6]:
# Configure the model parameters
model = DLinear_NN  # DLinear_NN, PatchTST_NN, GPT4TS_NN, CycleNet_NN, TimesNet_NN
model_params = {"moving_avg": 7, "individual": False}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

#### Scheduler with num_steps != num_epochs

Since users don’t have direct access to len(dataset), Tsururu provides custom relative parameters inside `scheduler_params` to correct schedulers' parameters:

- `relative_steps_per_epoch`: defines a multiplier for iterations in one epoch. Used for parameters like steps_per_epoch in OneCycleLR.
  - $absolute\_steps\_per\_epoch = relative\_steps\_per\_epoch * (len(dataset) // batch\_size + 1)$

- `relative_steps`: defines a multiplier of the total number of iterations across all epochs. Used for parameters, such as T_max in CosineAnnealingLR.
    - $absolute\_steps = relative\_steps * n\_epochs * (len(dataset) // batch\_size + 1)$

In [7]:
# 1) Default Scheduler (no custom parameters needed):
scheduler = torch.optim.lr_scheduler.StepLR
scheduler_after_epoch = True
scheduler_params = {
    "step_size": 2,
}

# 2) OneCycleLR using relative_steps_per_epoch:
scheduler = torch.optim.lr_scheduler.OneCycleLR
scheduler_after_epoch = False
scheduler_params = {
    "relative_steps_per_epoch": ["steps_per_epoch"],
    "steps_per_epoch": 1.0,
    "epochs": 10,
    "max_lr": 0.01,
}

# 3) 
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR
scheduler_after_epoch = False
scheduler_params = {
    "relative_steps": ["T_max"],
    "T_max": 1.0,
}

In [8]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    "save_to_dir": False,
    "n_epochs": 1,
    "scheduler": scheduler,
    "scheduler_after_epoch": scheduler_after_epoch,
    "scheduler_params": scheduler_params,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

### <a id='toc1_1_4_'></a>[Strategy](#toc0_)

In [9]:
horizon = 7
model_horizon = 7
history = 7

In [10]:
strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [11]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935
Updating learning rate to 0.001000.
Updating learning rate to 0.001000.
Updating learning rate to 0.000999.
Updating learning rate to 0.000998.
Updating learning rate to 0.000997.
Updating learning rate to 0.000996.
Updating learning rate to 0.000995.
Updating learning rate to 0.000993.
Updating learning rate to 0.000992.
Updating learning rate to 0.000990.
Updating learning rate to 0.000988.
Updating learning rate to 0.000985.
Updating learning rate to 0.000983.
Updating learning rate to 0.000980.
Updating learning rate to 0.000977.
Updating learning rate to 0.000974.
Updating learning rate to 0.000971.
Updating learning rate to 0.000967.
Updating learning rate to 0.000963.
Updating learning rate to 0.000959.
Updating learning rate to 0.000955.
Updating learning rate to 0.000951.
Updating learning rate to 0.000947.
Updating learning rate to 0.000942.
Updating learning rate to 0.000937.
Updating learning rate to 0.000932.
Updati

(11.653326034545898, <tsururu.strategies.mimo.MIMOStrategy at 0x1621f3cb0>)

In [12]:
forecast_time, current_pred = strategy.predict(dataset)

freq: Day; period: 1
length of test dataset: 10


In [13]:
current_pred

,id,date,value
0,0,2022-09-27,3599.11377
1,0,2022-09-28,3599.426025
2,0,2022-09-29,3599.793945
3,0,2022-09-30,3600.220215
4,0,2022-10-01,3600.378174
...,...,...,...
65,9,2022-09-29,8667.804688
66,9,2022-09-30,8668.47168
67,9,2022-10-01,8668.984375
68,9,2022-10-02,8669.640625


Saving and loading checkpoints is an essential practice in training DL models. 

Let's explore how to save checkpoints to disk, what structure the saved files have, and how to restore the model from a checkpoint for either fine-tuning or inference.

# <a id='toc2_'></a>[Save and load checkpoints](#toc0_)

Let’s consider working with checkpoints using the Direct strategy as an example.

In [14]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    # Let's enable save_to_dir (by the way, default value is True)
    "save_to_dir": True,
    "checkpoint_path": "checkpoints/",
    # Save checkpoints for 3 best model
    "save_k_best": 3,
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

### <a id='toc2_1_1_'></a>[Save checkpoint](#toc0_)

In [15]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935


Epoch 1/10, cost time: 2.65s
train loss: 4887171.9910
Validation, Loss: 1047792.3750, Metric: -1047792.3750
val loss: 1047792.3750, val metric: -1047792.3750
Epoch 2/10, cost time: 2.48s
train loss: 706722.3347
Validation, Loss: 462100.9062, Metric: -462100.9062
val loss: 462100.9062, val metric: -462100.9062
Epoch 3/10, cost time: 2.46s
train loss: 276882.1953
Validation, Loss: 154248.0312, Metric: -154248.0312
val loss: 154248.0312, val metric: -154248.0312
Epoch 4/10, cost time: 2.48s
train loss: 83515.5311
Validation, Loss: 39639.5547, Metric: -39639.5547
val loss: 39639.5547, val metric: -39639.5547
Epoch 5/10, cost time: 2.45s
train loss: 19590.2556
Validation, Loss: 8090.8174, Metric: -8090.8174
val loss: 8090.8174, val metric: -8090.8174
Epoch 6/10, cost time: 3.00s
train loss: 3694.6669
Validation, Loss: 1400.1285, Metric: -1400.1285
val loss: 1400.1285, val metric: -1400.1285
Epoch 7/10, cost time: 3.29s
train loss: 635.7974
Validation, Loss: 266.9762, Metric: -266.9762
val l

(104.16126585006714, <tsururu.strategies.mimo.MIMOStrategy at 0x168f70530>)

## <a id='toc2_2_'></a>[Load checkpoint for finetune](#toc0_)

Once we have the saved checkpoints, we can continue training by passing the pretrained path and another checkpoint path to the trainer’s parameters. All other parameters remain the same.

In [16]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "best_by_metric": True,
    # Let's enable save_to_dir (by the way, default value is True)
    "save_to_dir": True,
    "pretrained_path": "checkpoints/",
    "checkpoint_path": "checkpoints_finetuned/",
    # Save checkpoints for 3 best model
    "save_k_best": 3,
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [17]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935
Epoch 1/10, cost time: 2.58s
train loss: 16.7105
Validation, Loss: 11.4393, Metric: -11.4393
val loss: 11.4393, val metric: -11.4393
Epoch 2/10, cost time: 2.50s
train loss: 8.0148
Validation, Loss: 5.2676, Metric: -5.2676
val loss: 5.2676, val metric: -5.2676
Epoch 3/10, cost time: 2.49s
train loss: 3.5997
Validation, Loss: 2.2890, Metric: -2.2890
val loss: 2.2890, val metric: -2.2890
Epoch 4/10, cost time: 2.48s
train loss: 1.4931
Validation, Loss: 0.9049, Metric: -0.9049
val loss: 0.9049, val metric: -0.9049
Epoch 5/10, cost time: 2.50s
train loss: 0.5777
Validation, Loss: 0.3394, Metric: -0.3394
val loss: 0.3394, val metric: -0.3394
Epoch 6/10, cost time: 2.50s
train loss: 0.2106
Validation, Loss: 0.1200, Metric: -0.1200
val loss: 0.1200, val metric: -0.1200
Epoch 7/10, cost time: 2.49s
train loss: 0.0762
Validation, Loss: 0.0456, Metric: -0.0456
val loss: 0.0456, val metric: -0.0456
Epoch 8/10, cost time: 2.49s
train loss: 

(99.83380794525146, <tsururu.strategies.mimo.MIMOStrategy at 0x168b8ca70>)

## <a id='toc2_3_'></a>[Load checkpoint for inference](#toc0_)

In [18]:
trainer_params = {
    "device": "cpu",
    "num_workers": 0,
    "n_epochs": 1,
    "pretrained_path": "checkpoints_finetuned/",
    # Average checkpoints for the final model
    "average_snapshots": True,
    "verbose": False,
}

trainer = DLTrainer(
    model, 
    model_params, 
    validation, 
    validation_params, 
    **trainer_params
)

strategy = MIMOStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
)

In [19]:
strategy.fit(dataset)

length of train dataset: 4935
length of val dataset: 4935


Epoch 1/1, cost time: 2.55s
train loss: 0.0120
Validation, Loss: 0.0112, Metric: -0.0112
val loss: 0.0112, val metric: -0.0112
Fold 0. Score: -0.011183073744177818
length of train dataset: 4935
length of val dataset: 4935
Epoch 1/1, cost time: 2.49s
train loss: 0.0120
Validation, Loss: 0.0117, Metric: -0.0117
val loss: 0.0117, val metric: -0.0117
Fold 1. Score: -0.011746459640562534
Mean score: -0.0115
Std: 0.0003


(10.000695943832397, <tsururu.strategies.mimo.MIMOStrategy at 0x102fd17c0>)

In [20]:
forecast_time, current_pred = strategy.predict(dataset)

freq: Day; period: 1
length of test dataset: 10


In [21]:
current_pred

,id,date,value
0,0,2022-09-27,1999.970093
1,0,2022-09-28,2000.972412
2,0,2022-09-29,2001.976074
3,0,2022-09-30,2002.978027
4,0,2022-10-01,2003.981201
...,...,...,...
65,9,2022-09-29,11001.958984
66,9,2022-09-30,11002.960938
67,9,2022-10-01,11003.962891
68,9,2022-10-02,11004.966797
